In [44]:
import os
import random
import pandas as pd
from pytube.exceptions import VideoUnavailable
from pytube import YouTube
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

In [11]:
test_path = '01-03-2022-17_55_08_master.csv'
df = pd.read_csv(test_path)
df = df
df = df.reset_index()  # make sure indexes pair with number of rows
df.head()


,index,label,youtube_id,time_start,time_end,split
0,0,crowd,4pRPc04xoEo1,0,12,train
1,1,spray,4pRPc04xoEo2,12,18,train
2,2,running,4pRPc04xoEo3,12,24,train
3,3,crowd,4pRPc04xoEo4,34,44,train
4,4,crowd,4pRPc04xoEo5,66,71,train


In [54]:
df['label'].head(180).value_counts()

crowd               109
running              34
restraining          22
person_on_ground     18
striking             10
spray                 6
brawling              1
Name: label, dtype: int64

In [16]:
df['label'].value_counts()

spray               1145
striking            1145
aiming              1085
restraining         1078
throwing            1069
person_on_ground    1068
running              755
crowd                697
advancing            632
brawling             465
person_pointing      246
blood                 68
Name: label, dtype: int64

In [21]:
df['label'].value_counts()

spray               1145
striking            1145
aiming              1085
restraining         1078
throwing            1069
person_on_ground    1068
running              755
crowd                697
advancing            632
brawling             465
person_pointing      246
blood                 68
Name: label, dtype: int64

In [20]:
least = df.loc[df['label'] == 'brawling']
videos_per_split = len(least)*.35
videos_per_split

162.75

In [3]:
broken_videos = set()

def retry_download(tries, url, file_name, label):
    for i in range(tries):
            try:
                video = YouTube(url)
                yt_video = video.streams.get_highest_resolution()
                yt_video.download(output_path=f'videos/{label}/', filename=f"{file_name}.mp4")

                # download videos into correct folder
                ffmpeg_extract_subclip(filename=f'videos/{label}/{file_name}.mp4', t1=start, t2=end, targetname=f'videos/{label}/{file_name}_{start}_{end}.mp4') 
                os.remove(f'videos/{label}/{file_name}.mp4')
                
            except VideoUnavailable:
                if i < tries:
                    broken_videos.add(file_name)
                    continue

In [4]:
# Iterates all videos in csv
for index, row in df.iterrows():
    file_name = str(row['youtube_id'])
    start = int(row['time_start'])
    end = int(row['time_end'])
    label = str(row['label'])
    url = f'https://www.youtube.com/watch?v={file_name[0:11]}'
    fill_start = str(start).zfill(6)
    fill_end = str(end).zfill(6)

    if os.path.exists(f"videos/{label}/{file_name}_{start}_{end}.mp4"):
            print('file already exists')
            continue
    
    try:
        video = YouTube(url)
        yt_video = video.streams.get_highest_resolution()
        yt_video.download(output_path=f'videos/{label}/', filename=f"{file_name}.mp4")

        # download videos into correct folder
        
        ffmpeg_extract_subclip(filename=f'videos/{label}/{file_name}.mp4', t1=start, t2=end, 
                                targetname=f'videos/{label}/{file_name}_{fill_start}_{fill_end}.mp4') 
        os.remove(f'videos/{label}/{file_name}.mp4')

    except:
        print('retrying...', file_name)
        retry_download(1, url, file_name, label)      


Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join

In [22]:
# Deletes broken videos
for video in broken_videos:
    df = df.drop(df.loc[df['youtube_id'] == video].index)

In [23]:
# Adds working videos to list
working_videos = [i for i in df['youtube_id']]

In [33]:
with open('videos\label_map.txt', 'r') as f:
    label_list = [line.strip() for line in f]


In [ ]:
for label in df['label']:
    if label not in label_list:
        df = df.drop(df.loc[df['label'] == label].index)
    least_label_35 = int(df['label'].value_counts().min()*.35)
    

In [59]:
least_label_35 = int(df['label'].value_counts().min()*.35)
blah = df.groupby('label')['youtube_id'].apply(lambda s: s.sample(least_label_35))
    
    
    

In [64]:
blah

label         
brawling  7094     rs3rYkjA1mo2
          7245     xsdjaMLYmgk1
          7153     -11K0uXcIzM2
          7298     CXOvrl9m3wg4
          7371     r37J4gDLZ_c3
                      ...      
throwing  4036     IDFf66kSIrE5
          4131    CoWj-6NR4O029
          5670     yOhpEKWNN1U9
          5244    rrOjvhGoevY38
          5249    rrOjvhGoevY43
Name: youtube_id, Length: 1296, dtype: object

In [70]:
new_df = pd.DataFrame(blah)

In [71]:
new_df

youtube_id
label                       
brawling 7094   rs3rYkjA1mo2
         7245   xsdjaMLYmgk1
         7153   -11K0uXcIzM2
         7298   CXOvrl9m3wg4
         7371   r37J4gDLZ_c3
...                      ...
throwing 4036   IDFf66kSIrE5
         4131  CoWj-6NR4O029
         5670   yOhpEKWNN1U9
         5244  rrOjvhGoevY38
         5249  rrOjvhGoevY43

[1296 rows x 1 columns]